<a href="https://colab.research.google.com/github/KhurramDevOps/Quarter-02/blob/master/05_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploring GEMINI 2.0 Video and Audio Analysis :

In [1]:
!pip install --upgrade --quiet google-genai

In [2]:
!pip install gTTS --quiet

In [6]:
!pip install playsound --quiet

In [7]:
from google.colab import userdata
Video_process_API:str = userdata.get('GOOGLE_API_KEY')
if(Video_process_API):
  print("Api key found")
else:
  print("Api key not found")

Api key found


In [8]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key = Video_process_API)
model: str = "gemini-2.0-flash-exp"


In [9]:
# This appears to be a fle path and not python code
video_file_name = "/content/87C84A7E-2D28-4AD2-BB52-83BB6AB6A2F3.mp4"

In [23]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
    print("Waiting for video to be processed.")
    time.sleep(10)
    video_file = client.files.get(name= video_file.name or "")

  if video_file.state == "FAILED":
    raise Exception(f"Video processing failed: {video_file.error}")
  print(f"Video processing complete : " + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video processing complete : https://generativelanguage.googleapis.com/v1beta/files/t3wc20984fb9


In [25]:
from google.genai.types import Content, Part
prompt = """
for each scene in this video,
Generate captions that describe the scene along with any spoken text placed in quotation marks.
Place each caption into an object with the timecode of the caption in the video.
"""

video = my_video

response = client.models.generate_content(
    model= model,
    contents=[
        Content(
            role="User",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
      prompt,
    ]
)
print(response.text)


```json
[
  {
    "timecode": "0:00",
    "caption": "Two people are in a vehicle, and the person on the right is holding a stack of paper money. They are wearing dark sunglasses with the words \"I don't care\" on the lenses."
  },
  {
   "timecode": "0:01",
   "caption": "A man wearing a black jacket is holding a stack of paper money.  The person in the foreground is wearing dark sunglasses with the words “I don’t care” on the lenses and appears to be smiling. The text “Paisa paisa ho jayega bhai” is heard."
  },
  {
    "timecode": "0:02",
   "caption": "A hand is shown shuffling the stack of bills."
  },
  {
   "timecode": "0:03",
   "caption":"A person wearing a hijab and glasses with the words “I dont care” smiles at the camera. Someone behind them is holding money."
  },
    {
    "timecode": "0:04",
   "caption":"A man with glasses that say “I don’t care” is smiling. The background is blurry."
   },
   {
     "timecode": "0:05",
     "caption": "A man wearing a red shirt is smil

In [27]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text="My name is Khurram and im from lahore and im learning Agentic AI Now a days from PIAIC ", lang='en')


with open("output.mp3","wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

# Play the audio using IPython.display Audio to ensure it works in colab
# This is more reliable than playsound in Colab environments.

display(Audio("output.mp3",autoplay=True))
